In [1]:
import yaml
import pickle

with open("default_complete.yaml", 'r') as run_config: 
    data_config = yaml.load(run_config) 
with open("data/database_ml_parameters.yml", 'r') as param_config: 
    data_param = yaml.load(param_config) 
with open("data/config_model_parameters.yml", 'r') as mod_config: 
    data_model = yaml.load(mod_config) 
mcordata = "data"

indexp = 0
case = data_config["case"]
param_case = data_param[case]

In [2]:
from machine_learning_hep.listfiles import list_files_dir_lev2, list_files_lev2 
import multiprocessing as mp
import uproot

class Processer:
    # Class Attribute
    species = 'processer'

    # Initializer / Instance Attributes
    def __init__(self, datap, mcordata, indexp, maxfiles):
        
        self.mcordata = mcordata
        self.index_period = indexp
        self.maxfiles = maxfiles
        #namefile root
        self.n_root = datap["files_names"]["namefile_unmerged_tree"]
        #troot trees names
        self.n_treereco = datap["files_names"]["treeoriginreco"]
        self.n_treegen = datap["files_names"]["treeorigingen"]
        self.n_treeevt = datap["files_names"]["treeoriginevt"]
        #namefiles pkl
        self.n_reco = datap["files_names"]["namefile_reco"]
        self.n_evt = datap["files_names"]["namefile_evt"]
        self.n_gen = datap["files_names"]["namefile_gen"]
        #namefiles pkl skimmed
        self.n_recosk = datap["files_names"]["namefile_reco_skim"]
        self.n_evtsk = datap["files_names"]["namefile_evt_skim"]
        self.n_gensk = datap["files_names"]["namefile_gen_skim"]
        #directories
        self.d_root = datap["inputs"][self.mcordata]["unmerged_tree_dir"][self.index_period]
        self.d_reco = datap["output_folders"]["pkl_out"][self.mcordata][self.index_period]
        self.d_evt = self.d_reco
        self.d_gen = self.d_reco
        self.d_recosk = datap["output_folders"]["pkl_skimmed"][self.mcordata][self.index_period]
        self.d_evtsk = self.d_recosk
        self.d_recosk = self.d_recosk
        #selections
        self.s_reco_unp = datap["skimming_sel"]
        self.s_evt_unp = datap["skimming_sel_evt"]
        self.s_gen_unp = datap["skimming_sel_gen"]
        self.s_reco_skim = datap["skimming2_sel"]
        self.s_evt_skim = datap["skimming2_sel_evt"]
        self.s_gen_skim = datap["skimming2_sel_gen"]
        self.s_reco_trackpid = datap["skimming2_dotrackpid"]
        self.s_centr = datap["sel_cent"]
        #variables name
        self.v_all = datap["variables"]["var_all"]
        self.v_evt = datap["variables"]["var_evt"][self.mcordata]
        self.v_gen = datap["variables"]["var_all"]
        #list of files names
        self.l_root = None
        self.l_reco = None
        self.l_gen = None
        self.l_evt = None
        self.l_recosk = None
        self.l_gensk = None
        self.l_evtsk = None
        #parameter names
        self.maxperchunk = 30
        self.indexsample = None
        
    def set_maxperchunk(self, maxperchunk):
        self.maxperchunk = maxperchunk
    
    def buildlistpkl(self):
        self.l_root, self.l_reco = list_files_dir_lev2(self.d_root, self.d_reco, self.n_root, self.n_reco)
        _, self.l_gen = list_files_dir_lev2(self.d_root, self.d_gen, self.n_root, self.n_gen)
        _, self.l_evt = list_files_dir_lev2(self.d_root, self.d_evt, self.n_root, self.n_evt)

    def buildlistskim(self):
        _, self.l_recosk = list_files_dir_lev2(self.d_reco, self.d_recosk, self.n_reco, self.n_recosk)

    def unpack(self, file_index):
        print(self.l_root[file_index])
        treereco = uproot.open(self.l_root[file_index])[self.n_treereco]
        dfreco = treereco.pandas.df(branches=self.v_all)
        dfreco = dfreco.query(self.s_reco_unp)
        dfreco.to_pickle(self.l_reco[file_index])

        treeevt = uproot.open(self.l_root[file_index])[self.n_treeevt]
        dfevt = treeevt.pandas.df(branches=self.v_evt)
        if self.s_evt_unp is not None:
            dfevt.query(self.s_evt_unp)
        dfevt.to_pickle(self.l_evt[file_index])

        if (self.mcordata == "mc"):
            treegen = uproot.open(self.l_root[file_index])[self.n_treegen]
            dfgen = treegen.pandas.df(branches=self.v_gen)
            dfgen = dfreco.query(self.s_gen_unp)
            dfgen.to_pickle(self.l_gen[file_index])

    def skim(self, filein, fileout):
        df = pickle.load(open(filein, "rb"))
        df = df.query(self.s_reco_skim)
        df.to_pickle(fileout)
    
    def parallelizer(self, function, argument_list):
        chunks = [argument_list[x:x+self.maxperchunk] for x in range(0, len(argument_list), self.maxperchunk)]
        for chunk in chunks:
            pool = mp.Pool(self.maxperchunk)
            _ = [pool.apply(function,args=chunk[i]) for i in range(len(chunk))] 
            pool.close()

    def unpacker(self):
        self.buildlistpkl()
        arguments = [(i,) for i in range(len(self.l_root))]
        self.parallelizer(self.unpack, arguments)

#     def skimmer(self):
#         arguments = [(self.l_pkl[i], self.l_pklsk[i]) for i in range(len(self.l_pklsk))]
#         self.parallelizer(self.skim, arguments)

        
myprocess = Processer(data_param[case], mcordata, indexp, 10)
myprocess.unpacker()

dataroot/child_1/0029/AnalysisResults.root
dataroot/child_1/0028/AnalysisResults.root
dataroot/child_1/0026/AnalysisResults.root
dataroot/child_1/0030/AnalysisResults.root
dataroot/child_1/0024/AnalysisResults.root
dataroot/child_1/0023/AnalysisResults.root
dataroot/child_1/0022/AnalysisResults.root
dataroot/child_1/0025/AnalysisResults.root
